# 1. Introduction

Nikki Satmaka - Batch 11

## Description

Dataset is taken from [Kaggle](https://www.kaggle.com/blastchar/telco-customer-churn)

Context:

This dataset contains the information related to customers of a telco company that provided home phone and Internet services to 7043 customers in California in Q3. It indicates which customers have left or stayed for their service.

1. `customerID` : A unique ID that identifies each customer.
1. `gender` : The customer’s gender: Male, Female
1. `SeniorCitizen` : Indicates if the customer is 65 or older: 1 = Senior Citizen, 0 = Not Senior Citizen
1. `Partner` : Indicates if the customer is married: Yes, No
1. `Dependents` : Indicates if the customer lives with any dependents: Yes, No. Dependents could be children, parents, grandparents, etc.
1. `tenure` :  Indicates the total amount of months that the customer has been with the company.
1. `PhoneService` : Indicates if the customer subscribes to home phone service with the company: Yes, No
1. `MultipleLines` : Indicates if the customer subscribes to multiple telephone lines with the company: Yes, No, No phone service
1. `InternetService` : Indicates if the customer subscribes to Internet service with the company: No, DSL, Fiber Optic
1. `OnlineSecurity` : Indicates if the customer subscribes to an additional online security service provided by the company: Yes, No, No internet service
1. `OnlineBackup` : Indicates if the customer subscribes to an additional online backup service provided by the company: Yes, No, No internet service
1. `DeviceProtection` : Indicates if the customer subscribes to an additional device protection plan for their Internet equipment provided by the company: Yes, No, No internet service
1. `TechSupport` : Indicates if the customer subscribes to an additional technical support plan from the company with reduced wait times: Yes, No, No internet service
1. `StreamingTV` : Indicates if the customer uses their Internet service to stream television programing from a third party provider: Yes, No, No internet service
1. `StreamingMovies` : Indicates if the customer uses their Internet service to stream movies from a third party provider: Yes, No, No internet service
1. `Contract` : Indicates the customer’s current contract type: Month-to-Month, One Year, Two Year.
1. `PaperlessBilling` : Indicates if the customer has chosen paperless billing: Yes, No
1. `PaymentMethod` : Indicates how the customer pays their bill: Bank Transfer, Credit Card, Electronic Check, Mailed Check
1. `MonthlyCharges` : Indicates the customer’s current total monthly charge for all their services from the company.
1. `TotalCharges` : Indicates the customer’s total charges.
1. `Churn` : Yes = the customer left the company. No = the customer remained with the company.

### Objective

- Predict whether a customer will churn and stop using the services

### Problem Statement

- What features are going to be useful as predictors?

# 2. Importing Libraries

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import joblib

import warnings
warnings.filterwarnings('ignore')

# For Data Preprocessing
from imblearn.over_sampling import SMOTENC
from sklearn.compose import ColumnTransformer

# Split Dataset and Standarize the Datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# Neural Network
import tensorflow as tf
from tensorflow import keras

# Evaluate Classification Models
from sklearn.metrics import classification_report

# # Useful functions
# from packages.checker import check_missing
# from packages.outlier_handling import outlier_summary
# from packages.imputation_handling import impute_total_charges
# from packages.imputation_handling import impute_no_phone_internet
# from packages.visualization import kdeplot, plot_loss, plot_acc

pd.set_option('display.precision', 2)

sns.set_theme(style='darkgrid', palette='Set1')

# set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline

# 3. Data Loading

In [2]:
# load application dataset
df_ori = pd.read_csv('data/app_train.csv')
df_ori = df_ori.drop(['Unnamed: 0'], axis=1)
# make column names lowercase
df_ori.columns = df_ori.columns.str.lower()
app_train = df_ori.copy()

# load previous application dataset
prev = pd.read_csv('data/prev_app.csv')
prev = prev.drop(['Unnamed: 0'], axis=1)
prev.columns = prev.columns.str.lower()

# load installment dataset
installment = pd.read_csv('data/installment_payment.csv')
installment = installment.drop(['Unnamed: 0'], axis=1)
installment.columns = installment.columns.str.lower()

# display the first 5 entries of the data
app_train.head()

,ln_id,target,contract_type,gender,num_children,income,approved_credit,annuity,price,income_type,...,days_age,days_work,days_registration,days_id_change,weekdays_apply,hour_apply,organization_type,ext_score_1,ext_score_2,ext_score_3
0,333538,0,Revolving loans,F,1,67500.0,202500.0,10125.0,202500.0,Working,...,-11539,-921,-119.0,-2757,TUESDAY,18,Business Entity Type 3,0.57,0.61,NaN
1,406644,0,Cash loans,F,1,202500.0,976711.5,49869.0,873000.0,Commercial associate,...,-15743,-4482,-1797.0,-2455,TUESDAY,14,Other,0.66,0.68,NaN
2,259130,0,Cash loans,F,0,180000.0,407520.0,25060.5,360000.0,Pensioner,...,-20775,365243,-8737.0,-4312,THURSDAY,14,NA1,NaN,0.58,0.75
3,411997,0,Cash loans,M,0,225000.0,808650.0,26086.5,675000.0,State servant,...,-20659,-10455,-4998.0,-4010,WEDNESDAY,10,Culture,NaN,0.62,0.71
4,241559,0,Revolving loans,M,0,135000.0,180000.0,9000.0,180000.0,Commercial associate,...,-9013,-1190,-3524.0,-1644,SUNDAY,11,Construction,0.18,0.49,0.09


In [3]:
app_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61503 entries, 0 to 61502
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ln_id              61503 non-null  int64  
 1   target             61503 non-null  int64  
 2   contract_type      61503 non-null  object 
 3   gender             61503 non-null  object 
 4   num_children       61503 non-null  int64  
 5   income             61503 non-null  float64
 6   approved_credit    61503 non-null  float64
 7   annuity            61502 non-null  float64
 8   price              61441 non-null  float64
 9   income_type        61503 non-null  object 
 10  education          61503 non-null  object 
 11  family_status      61503 non-null  object 
 12  housing_type       61503 non-null  object 
 13  days_age           61503 non-null  int64  
 14  days_work          61503 non-null  int64  
 15  days_registration  61503 non-null  float64
 16  days_id_change     615

In [4]:
prev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350712 entries, 0 to 350711
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   sk_id_prev                 350712 non-null  int64  
 1   ln_id                      350712 non-null  int64  
 2   contract_type              350712 non-null  object 
 3   annuity                    274103 non-null  float64
 4   application                350712 non-null  float64
 5   approved_credit            350712 non-null  float64
 6   amt_down_payment           164205 non-null  float64
 7   price                      271072 non-null  float64
 8   weekdays_apply             350712 non-null  object 
 9   hour_apply                 350712 non-null  int64  
 10  contract_status            350712 non-null  object 
 11  days_decision              350712 non-null  int64  
 12  term_payment               274103 non-null  float64
 13  yield_group                35

In [5]:
installment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2872306 entries, 0 to 2872305
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   sk_id_prev   int64  
 1   ln_id        int64  
 2   inst_number  int64  
 3   inst_days    float64
 4   pay_days     float64
 5   amt_inst     float64
 6   amt_pay      float64
dtypes: float64(4), int64(3)
memory usage: 153.4 MB


In [6]:
app_train.iloc[0]

ln_id                                       333538
target                                           0
contract_type                      Revolving loans
gender                                           F
num_children                                     1
income                                     67500.0
approved_credit                           202500.0
annuity                                    10125.0
price                                     202500.0
income_type                                Working
education            Secondary / secondary special
family_status                              Married
housing_type                          With parents
days_age                                    -11539
days_work                                     -921
days_registration                           -119.0
days_id_change                               -2757
weekdays_apply                             TUESDAY
hour_apply                                      18
organization_type           Bus

In [7]:
prev.head()

,sk_id_prev,ln_id,contract_type,annuity,application,approved_credit,amt_down_payment,price,weekdays_apply,hour_apply,contract_status,days_decision,term_payment,yield_group,first_draw,first_due,termination,nflag_insured_on_approval
0,2030495,271877,Consumer loans,1730.43,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Approved,-73,12.0,middle,365243.0,-42.0,-37.0,0.0
1,2819243,176158,Cash loans,47041.33,450000.0,470790.0,NaN,450000.0,MONDAY,7,Approved,-512,12.0,middle,365243.0,-482.0,-177.0,1.0
2,1383531,199383,Cash loans,23703.93,315000.0,340573.5,NaN,315000.0,SATURDAY,8,Approved,-684,18.0,low_normal,365243.0,-654.0,-137.0,1.0
3,2315218,175704,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,Canceled,-14,NaN,NA1,NaN,NaN,NaN,NaN
4,1715995,447712,Cash loans,11368.62,270000.0,335754.0,NaN,270000.0,FRIDAY,7,Approved,-735,54.0,low_normal,365243.0,-705.0,-334.0,1.0


In [8]:
# create a list of all new ln_id
new_ln_id = app_train[~app_train['ln_id'].isin(prev[prev['ln_id'].isin(app_train['ln_id'].unique())]['ln_id'])]['ln_id'].values

In [9]:
prev['contract_type'].unique()

array(['Consumer loans', 'Cash loans', 'Revolving loans', 'NA1'],
      dtype=object)

In [12]:
prev['contract_status'].unique()

array(['Approved', 'Canceled', 'Refused', 'Unused offer'], dtype=object)

In [14]:
prev[prev['contract_type'] == 'NA1']['contract_status'].unique()

array(['Canceled', 'Refused'], dtype=object)

In [33]:
app_train[app_train['ln_id'].isin(prev[prev['contract_status'] == 'Canceled']['ln_id'].unique())]['target'].value_counts()

0    20325
1     1917
Name: target, dtype: int64

In [22]:
prev[prev['contract_status'] == 'Refused']

,sk_id_prev,ln_id,contract_type,annuity,application,approved_credit,amt_down_payment,price,weekdays_apply,hour_apply,contract_status,days_decision,term_payment,yield_group,first_draw,first_due,termination,nflag_insured_on_approval
12,2012211,282125,Revolving loans,33750.00,0.00e+00,6.75e+05,NaN,NaN,FRIDAY,17,Refused,-568,0.0,NA1,NaN,NaN,NaN,NaN
28,1892110,372945,Consumer loans,12607.69,7.56e+04,6.81e+04,7564.1,7.56e+04,TUESDAY,10,Refused,-2654,6.0,middle,NaN,NaN,NaN,NaN
50,2724439,163961,Cash loans,39369.60,3.60e+05,3.60e+05,0.0,3.60e+05,WEDNESDAY,12,Refused,-2337,12.0,high,NaN,NaN,NaN,NaN
52,1656819,324462,Consumer loans,NaN,1.13e+05,1.13e+05,NaN,1.13e+05,THURSDAY,13,Refused,-1075,NaN,NA1,NaN,NaN,NaN,NaN
53,1741220,181080,Revolving loans,6750.00,1.35e+05,1.35e+05,NaN,1.35e+05,SATURDAY,12,Refused,-538,0.0,NA1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350703,2405492,154531,Cash loans,NaN,0.00e+00,0.00e+00,NaN,NaN,FRIDAY,11,Refused,-200,NaN,NA1,NaN,NaN,NaN,NaN
350704,2656210,223981,Cash loans,26316.00,9.00e+05,9.00e+05,NaN,9.00e+05,THURSDAY,17,Refused,-32,60.0,low_normal,NaN,NaN,NaN,NaN
350706,1229066,237195,Cash loans,18534.51,2.25e+05,2.55e+05,NaN,2.25e+05,WEDNESDAY,9,Refused,-376,24.0,high,NaN,NaN,NaN,NaN
350707,1379569,309506,Cash loans,33389.10,1.04e+06,1.04e+06,NaN,1.04e+06,THURSDAY,10,Refused,-156,60.0,low_normal,NaN,NaN,NaN,NaN


In [ ]:
np.array_equal(np.sort(app_train['ln_id'].unique()), np.sort(prev['ln_id'].unique()))

In [ ]:
prev[prev['ln_id'].isin(app_train['ln_id'].unique())]['ln_id'].nunique()

In [ ]:
app_train['ln_id'].nunique()

In [ ]:
prev['ln_id'].nunique()

In [ ]:
# display the last 5 entries of the data
df.tail()

## Data Understanding

In [ ]:
# check dataset shape
df.shape

There are 7043 instances and 21 columns of data

In [ ]:
# check dataset info
df.info()

This is weird. Why would `TotalCharges` have `object` as a dtype? It should be similar to `MonthlyCharges` which is `float64`\
Let's see the unique values

In [ ]:
# check unique values for `TotalCharges` feature
print(df['TotalCharges'].unique())

Glancing at the unique values, it's clear that they're numbers, but quoted as string.\
Let's convert `TotalCharges` to `number` and pass any errors as `NaN`

In [ ]:
# convert `TotalCharges` to numeric and pass any non-numeric values as NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

Let's check the info again

In [ ]:
# check dataset info
df.info()

In [ ]:
# check duplicate values in dataset
df[df.duplicated()]

Great! There are no duplicated instances

In [ ]:
# check missing values in dataset
check_missing(df)

Seems like those 11 missing values were the cause of that `object` data type\
We've got 0.16% missing values in `TotalCharges`

## Basic Characteristics of the Dataset

In [ ]:
# check basic stats for numerical features
df.describe(percentiles=[0.5]).T

In [ ]:
# check basic stats for features with object dtypes
df.describe(include='object').T

In [ ]:
# check the cardinality of each nominal and ordinal features
print("Nominal and ordinal features cardinality:")
for col in df.columns:
    if df[col].nunique() < 20:
        print(col, ':', df[col].nunique(), 'unique values \n', np.sort(df[col].unique()))
        print('-' * 100)

Let's cover some basic stats of the features in the train set.
- 

## Data Preparation

Drop `customerID` because it's an index

In [ ]:
# drop `customerID` from the dataset
df = df.drop(['customerID'], axis=1)

## Check for Dataset Imbalance

Check whether the target variable of the dataset is balance

In [ ]:
# check for imbalance in target variable
plt.figure(figsize=(4,5))
sns.countplot(data=df, x='Churn')
plt.title('Number of customers churned')
plt.xlabel(None)
plt.ylabel(None)
plt.ylim(0, df.shape[0])

plt.show()

We can see that most customers do not churn and the percentage of churned clients is less than 30% of the dataset.\
That means our dataset is imbalance and we need to stratify when splitting

## Splitting Dataset

We need to split the dataset into train and test sets before we do any EDA.\
We do our EDA on the train set so as to not have any bias towards the whole dataset.

### Split train and test set


Since the target variable is imbalanced, we use stratified sampling

In [ ]:
# we use stratified sampling to ensure that the distribution of the target variable is balanced
df_train_valid, df_test = train_test_split(
    df,
    test_size=0.20,
    random_state=42,
    stratify=df['Churn']
)

print('df_train_valid Size:', df_train_valid.shape)
print('df_test Size:', df_test.shape)

### Split train and validation set


Since the target variable is imbalanced, we use stratified sampling

In [ ]:
# we use stratified sampling to ensure that the distribution of the target variable is balanced
df_train, df_valid = train_test_split(
    df_train_valid,
    test_size=0.20,
    random_state=42,
    stratify=df_train_valid['Churn']
)

print('df_train Size:', df_train.shape)
print('df_valid Size:', df_valid.shape)

In [ ]:
# backup the train set that we are gonna perform EDA on
df_train_ori = df_train.copy()

# 4. Exploratory Data Analysis

## Subheading 1